## Portfolio assignment 16
30 min: Train a decision tree to predict one of the categorical columns of your own dataset.
- Split your dataset into a train (70%) and test (30%) set.
- Use the train set to fit a DecisionTreeClassifier. You are free to to choose which columns you want to use as feature variables and you are also free to choose the max_depth of the tree. 
- Use your decision tree model to make predictions for both the train and test set.
- Calculate the accuracy for both the train set predictions and test set predictions.
- Is the accurracy different? Did you expect this difference?
- Use the plot_tree function above to create a plot of the decision tree. Take a few minutes to analyse the decision tree. Do you understand the tree?

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
steam = pd.read_csv('../week 1/steam_games.csv')
steam.head()

FileNotFoundError: [Errno 2] No such file or directory: '../week 1/steam_games.csv'

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
len(steam)

In [ ]:
len(steam.dropna())

that's a lot less rows... let's fill up the empty spaces. <br>
we'll start by filling up the numerical values

In [ ]:
steam = steam.fillna(value={'achievements': 0, 'discount_price': steam.original_price})

In [ ]:
len(steam.dropna(subset=['developer']))

This is still a fine size, plus it's weird to have games without a developer.

In [ ]:
steam = steam.dropna(subset=['developer'])

In [ ]:
features= ['achievements']
dt = DecisionTreeClassifier(max_depth = 1) # Increase max_depth to see effect in the plot
dt.fit(steam[features], steam['developer'])

In [ ]:
from sklearn import tree
import graphviz

def plot_tree_classification(model, features, class_names):
    # Generate plot data
    dot_data = tree.export_graphviz(model, out_file=None, 
                          feature_names=features,  
                          class_names=class_names,  
                          filled=True, rounded=True,  
                          special_characters=True)  

    # Turn into graph using graphviz
    graph = graphviz.Source(dot_data)  

    # Write out a pdf
    graph.render("decision_tree")

    # Display in the notebook
    return graph 

In [ ]:
plot_tree_classification(dt, features, steam.developer.unique())

Here it asks if there are less than 0.5 achievements, it'll be from Tokiwa Graphics.

In [ ]:
predictions = dt.predict(steam[features])

In [ ]:
def calculate_accuracy(predictions, actuals):
    if(len(predictions) != len(actuals)):
        raise Exception("The amount of predictions did not equal the amount of actuals")
    
    return (predictions == actuals).sum() / len(actuals)

In [ ]:
calculate_accuracy(predictions, steam.developer)

I guess this has a very low accuracy.

In [ ]:
len(steam.developer.unique())

This might be why

In [ ]:
top10 = steam.developer.value_counts().sort_values(ascending=False).index[:10]

In [ ]:
steam.loc[~steam.developer.isin(top10),'developer'] = 'Other'

In [ ]:
len(steam.developer.unique())

In [ ]:
steam.developer.value_counts()

In [ ]:
features= ['achievements']
dt = DecisionTreeClassifier(max_depth = 10) # Increase max_depth to see effect in the plot
dt.fit(steam[features], steam['developer'])

In [ ]:
plot_tree_classification(dt, features, steam.developer.unique())

Apparently now the majority is Ubisoft, so we're going to have to check per class.